# Env Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
%cd /content/drive/MyDrive/5370_Project_E2E/E2E-DRO/
try: import alpha_vantage
except ImportError: get_ipython().system('pip install alpha_vantage')

try: import cvxpylayers
except ImportError: get_ipython().system('pip install cvxpylayers')

import numpy as np
if np.__version__ != '1.25.2':
  get_ipython().system('pip install --upgrade numpy==1.25.2')

try:
  import pandas as pd
  if pd.__version__ != '1.5.3':
    get_ipython().system('pip install --upgrade pandas==1.5.3')
except ImportError: get_ipython().system('pip install pandas==1.5.3')

import matplotlib
if matplotlib.__version__ != '3.5.1':
  get_ipython().system('pip install --upgrade matplotlib==3.5.1')

import os
# os.kill(os.getpid(), 9)

import matplotlib as mpl
mpl.use('Agg')  # Non-interactive backend that doesn't require TeX
mpl.rcParams['text.usetex'] = False
mpl.rcParams['pdf.fonttype'] = 42



/content/drive/MyDrive/5370_Project_E2E/E2E-DRO


# Redo DR Experiment

In [ ]:
%cd /content/drive/MyDrive/5370_Project_E2E/E2E-DRO/
import warnings
warnings.filterwarnings("ignore")
#%run main.py

/content/drive/MyDrive/5370_Project_E2E/E2E-DRO


In [ ]:
# Distributionally Robust End-to-End Portfolio Construction
# Experiment 1 - General
####################################################################################################
# Import libraries
####################################################################################################
import torch
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
plt.close("all")
plt.rcParams['text.usetex'] = False


# Make the code device-agnostic
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Import E2E_DRO functions
from e2edro import e2edro as e2e
from e2edro import DataLoad as dl
from e2edro import BaseModels as bm
from e2edro import PlotFunctions as pf

# Path to cache the data, models and results
cache_path = "./cache/exp/"

####################################################################################################
# Experiments 1-4 (with hisotrical data): Load data
####################################################################################################

# Data frequency and start/end dates
freq = 'weekly'
start = '2000-01-01'
end = '2021-09-30'

# Train, validation and test split percentage
split = [0.6, 0.4]

# Number of observations per window
n_obs = 104

# Number of assets
n_y = 20

# AlphaVantage API Key.
# Note: User API keys can be obtained for free from www.alphavantage.co. Users will need a free
# academic or paid license to download adjusted closing pricing data from AlphaVantage.
AV_key = None

# Historical data: Download data (or load cached data)
X, Y = dl.AV(start, end, split, freq=freq, n_obs=n_obs, n_y=n_y, use_cache=True,
            save_results=False, AV_key=AV_key)

# Number of features and assets
n_x, n_y = X.data.shape[1], Y.data.shape[1]

# Statistical significance analysis of features vs targets
stats = dl.statanalysis(X.data, Y.data)

####################################################################################################
# E2E Learning System Run
####################################################################################################

#---------------------------------------------------------------------------------------------------
# Initialize parameters
#---------------------------------------------------------------------------------------------------

# Performance loss function and performance period 'v+1'
perf_loss='sharpe_loss'
perf_period = 13

# Weight assigned to MSE prediction loss function
pred_loss_factor = 0.5

# Risk function (default set to variance)
prisk = 'p_var'

# Robust decision layer to use: hellinger or tv
dr_layer = 'hellinger'

# List of learning rates to test
lr_list = [0.005, 0.0125, 0.02]

# List of total no. of epochs to test
epoch_list = [30, 40, 50, 60, 80, 100]

# For replicability, set the random seed for the numerical experiments
set_seed = 1000

# Load saved models (default is False)
use_cache = True



In [ ]:
#---------------------------------------------------------------------------------------------------
# Run
#---------------------------------------------------------------------------------------------------

if use_cache:
    # Load cached models and backtest results
    with open(cache_path+'ew_net.pkl', 'rb') as inp:
        ew_net = pickle.load(inp)
    with open(cache_path+'po_net.pkl', 'rb') as inp:
        po_net = pickle.load(inp)
    with open(cache_path+'base_net.pkl', 'rb') as inp:
        base_net = pickle.load(inp)
    with open(cache_path+'nom_net.pkl', 'rb') as inp:
        nom_net = pickle.load(inp)
    with open(cache_path+'dr_net.pkl', 'rb') as inp:
        dr_net = pickle.load(inp)
    with open(cache_path+'dr_po_net.pkl', 'rb') as inp:
        dr_po_net = pickle.load(inp)
    with open(cache_path+'dr_net_learn_delta.pkl', 'rb') as inp:
        dr_net_learn_delta = pickle.load(inp)
    with open(cache_path+'nom_net_learn_gamma.pkl', 'rb') as inp:
        nom_net_learn_gamma = pickle.load(inp)
    with open(cache_path+'dr_net_learn_gamma.pkl', 'rb') as inp:
        dr_net_learn_gamma = pickle.load(inp)
    with open(cache_path+'dr_net_learn_gamma_delta.pkl', 'rb') as inp:
        dr_net_learn_gamma_delta = pickle.load(inp)
    with open(cache_path+'nom_net_learn_theta.pkl', 'rb') as inp:
        nom_net_learn_theta = pickle.load(inp)
    with open(cache_path+'dr_net_learn_theta.pkl', 'rb') as inp:
        dr_net_learn_theta = pickle.load(inp)

    with open(cache_path+'base_net_ext.pkl', 'rb') as inp:
        base_net_ext = pickle.load(inp)
    with open(cache_path+'nom_net_ext.pkl', 'rb') as inp:
        nom_net_ext = pickle.load(inp)
    with open(cache_path+'dr_net_ext.pkl', 'rb') as inp:
        dr_net_ext = pickle.load(inp)
    with open(cache_path+'dr_net_learn_delta_ext.pkl', 'rb') as inp:
        dr_net_learn_delta_ext = pickle.load(inp)
    with open(cache_path+'nom_net_learn_gamma_ext.pkl', 'rb') as inp:
        nom_net_learn_gamma_ext = pickle.load(inp)
    with open(cache_path+'dr_net_learn_gamma_ext.pkl', 'rb') as inp:
        dr_net_learn_gamma_ext = pickle.load(inp)
    with open(cache_path+'nom_net_learn_theta_ext.pkl', 'rb') as inp:
        nom_net_learn_theta_ext = pickle.load(inp)
    with open(cache_path+'dr_net_learn_theta_ext.pkl', 'rb') as inp:
        dr_net_learn_theta_ext = pickle.load(inp)

    with open(cache_path+'dr_net_tv.pkl', 'rb') as inp:
        dr_net_tv = pickle.load(inp)
    with open(cache_path+'dr_net_tv_learn_delta.pkl', 'rb') as inp:
        dr_net_tv_learn_delta = pickle.load(inp)
    with open(cache_path+'dr_net_tv_learn_gamma.pkl', 'rb') as inp:
        dr_net_tv_learn_gamma = pickle.load(inp)
    with open(cache_path+'dr_net_tv_learn_theta.pkl', 'rb') as inp:
        dr_net_tv_learn_theta = pickle.load(inp)
else:
    # Exp 1: Equal weight portfolio
    ew_net = bm.equal_weight(n_x, n_y, n_obs)
    ew_net.net_roll_test(X, Y, n_roll=4)
    with open(cache_path+'ew_net.pkl', 'wb') as outp:
            pickle.dump(ew_net, outp, pickle.HIGHEST_PROTOCOL)
    print('ew_net run complete')

    # Exp 1, 2, 3: Predict-then-optimize system
    po_net = bm.pred_then_opt(n_x, n_y, n_obs, set_seed=set_seed, prisk=prisk).double()
    po_net.net_roll_test(X, Y)
    with open(cache_path+'po_net.pkl', 'wb') as outp:
        pickle.dump(po_net, outp, pickle.HIGHEST_PROTOCOL)
    print('po_net run complete')

    # Exp 1: Base E2E
    base_net = e2e.e2e_net(n_x, n_y, n_obs, prisk=prisk,
                        train_pred=True, train_gamma=False, train_delta=False,
                        set_seed=set_seed, opt_layer='base_mod', perf_loss=perf_loss,
                        perf_period=perf_period, pred_loss_factor=pred_loss_factor).double()
    base_net.net_cv(X, Y, lr_list, epoch_list)
    base_net.net_roll_test(X, Y)
    with open(cache_path+'base_net.pkl', 'wb') as outp:
        pickle.dump(base_net, outp, pickle.HIGHEST_PROTOCOL)
    print('base_net run complete')

    # Exp 1: Nominal E2E
    nom_net = e2e.e2e_net(n_x, n_y, n_obs, prisk=prisk,
                        train_pred=True, train_gamma=True, train_delta=False,
                        set_seed=set_seed, opt_layer='nominal', perf_loss=perf_loss,
                        cache_path=cache_path, perf_period=perf_period,
                        pred_loss_factor=pred_loss_factor).double()
    nom_net.net_cv(X, Y, lr_list, epoch_list)
    nom_net.net_roll_test(X, Y)
    with open(cache_path+'nom_net.pkl', 'wb') as outp:
        pickle.dump(nom_net, outp, pickle.HIGHEST_PROTOCOL)
    print('nom_net run complete')

    # Exp 1: DR E2E
    dr_net = e2e.e2e_net(n_x, n_y, n_obs, prisk=prisk,
                        train_pred=True, train_gamma=True, train_delta=True,
                        set_seed=set_seed, opt_layer=dr_layer, perf_loss=perf_loss,
                        cache_path=cache_path, perf_period=perf_period,
                        pred_loss_factor=pred_loss_factor).double()
    dr_net.net_cv(X, Y, lr_list, epoch_list)
    dr_net.net_roll_test(X, Y)
    with open(cache_path+'dr_net.pkl', 'wb') as outp:
        pickle.dump(dr_net, outp, pickle.HIGHEST_PROTOCOL)
    print('dr_net run complete')


####################################################################################################
# Merge objects with their extended-epoch counterparts
####################################################################################################
if use_cache:
    portfolios = ["base_net", "nom_net", "dr_net", "dr_net_learn_delta", "nom_net_learn_gamma",
                "dr_net_learn_gamma", "nom_net_learn_theta", "dr_net_learn_theta"]

    for portfolio in portfolios:
        cv_combo = pd.concat([eval(portfolio).cv_results, eval(portfolio+'_ext').cv_results],
                        ignore_index=True)
        eval(portfolio).load_cv_results(cv_combo)
        if eval(portfolio).epochs > 50:
            exec(portfolio + '=' + portfolio+'_ext')
            eval(portfolio).load_cv_results(cv_combo)

####################################################################################################
# Numerical results
####################################################################################################


# Validation results table
dr_net.cv_results = dr_net.cv_results.sort_values(['epochs', 'lr'],
                                                  ascending=[True, True]
                                                  ).reset_index(drop=True)
exp1_validation_table = pd.concat((base_net.cv_results.round(4),
                            nom_net.cv_results.val_loss.round(4),
                            dr_net.cv_results.val_loss.round(4)), axis=1)
exp1_validation_table.set_axis(['eta', 'Epochs', 'Base', 'Nom.', 'DR'],
                        axis=1, inplace=True)

plt.rcParams['text.usetex'] = True
portfolio_names = [r'EW', r'PO', r'Base', r'Nominal', r'DR']
portfolios = [ew_net.portfolio,
              po_net.portfolio,
              base_net.portfolio,
              nom_net.portfolio,
              dr_net.portfolio]

# Out-of-sample summary statistics table
exp1_fin_table = pf.fin_table(portfolios, portfolio_names)

# Wealth evolution plot
portfolio_colors = ["dimgray",
                    "forestgreen",
                    "goldenrod",
                    "dodgerblue",
                    "salmon"]
port_realized_r = pf.wealth_plot(portfolios, portfolio_names, portfolio_colors,
                path=cache_path+"plots/wealth_exp1.png")
port_sharp_r = pf.sr_bar(portfolios, portfolio_names, portfolio_colors,
                path=cache_path+"plots/sr_bar_exp1.png")

# List of initial parameters
exp1_param_dict = dict({'po_net':po_net.gamma.item(),
                'nom_net':nom_net.gamma_init,
                'dr_net':[dr_net.gamma_init, dr_net.delta_init]})

# Trained values for each out-of-sample investment period
exp1_trained_vals = pd.DataFrame(zip([nom_net.gamma_init]+nom_net.gamma_trained,
                                    [dr_net.gamma_init]+dr_net.gamma_trained,
                                    [dr_net.delta_init]+dr_net.delta_trained),
                                    columns=[r'Nom. gamma',
                                             r'DR gamma',
                                             r'DR delta'])

# Our Models:

In [ ]:
import numpy as np
import torch
import e2edro.PortfolioClasses as pc

def non_overlapping_equal_weight_test(X, Y, n_assets=None, portfolio_constructor=None, holding_period=14):
    """
    Tests performance of an equal-weighted portfolio using non-overlapping periods.
    Rebalances at the start of each holding period and tracks daily performance.

    Parameters:
    -----------
    X : object
        Features object with test attribute
    Y : object
        Target/returns object with test attribute
    n_assets : int, optional
        Number of assets in the portfolio. If None, will be inferred from Y shape.
    portfolio_constructor : module, optional
        Module with backtest functionality. If None, will attempt to import.
    holding_period : int, optional
        Number of days to hold the portfolio before rebalancing

    Returns:
    --------
    portfolio : object
        Portfolio object with weights, returns, and performance metrics
    """
    # Get test data
    X_test = X.test
    Y_test = Y.test

    # Convert to tensors
    Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.float32)

    # Get test dates
    test_dates = Y_test.index

    # Calculate how many non-overlapping periods fit in the test set
    available_days = len(test_dates)
    num_periods = available_days // holding_period

    # Total number of days we'll track in our portfolio
    total_days = num_periods * holding_period

    # Make sure we don't exceed available data
    if total_days > available_days:
        total_days = available_days

    # Get or infer the number of assets
    if n_assets is None:
        n_assets = Y_test.shape[1]

    # Get or import the portfolio construction modul
    # Create portfolio object for storing daily results
    portfolio = pc.backtest(total_days - 1, n_assets, test_dates[:total_days - 1])

    # Arrays to store weights and daily returns
    all_weights = np.zeros((total_days - 1, n_assets))
    daily_returns = np.zeros(total_days - 1)

    print(f"Testing equal-weighted portfolio with {holding_period}-day rebalancing across {num_periods} periods")

    # Initialize portfolio value series (starting at 1.0)
    portfolio_values = np.ones(total_days)

    # Equal weights (1/N for each asset)
    equal_weights = np.ones(n_assets) / n_assets

    with torch.no_grad():
        # For each non-overlapping holding period
        for period in range(num_periods):
            # Calculate the starting index for this period
            period_start_idx = period * holding_period
            period_end_idx = min(period_start_idx + holding_period, total_days)

            print(f"Processing period {period+1}/{num_periods} (days {period_start_idx}-{period_end_idx-1})")

            try:
                # Store weights for each day in the holding period
                for t in range(period_start_idx, period_end_idx - 1):  # -1 because we need next day's data
                    all_weights[t] = equal_weights

                # Calculate daily returns during the holding period
                for t in range(period_start_idx, period_end_idx - 1):
                    # Calculate daily return from t to t+1
                    curr_prices = Y_test_tensor[t]
                    next_prices = Y_test_tensor[t+1]
                    daily_price_returns = (next_prices - curr_prices) / curr_prices

                    # Portfolio daily return
                    weights_tensor = torch.tensor(equal_weights,
                                                 dtype=daily_price_returns.dtype,
                                                 device=daily_price_returns.device)

                    daily_return = float(torch.sum(weights_tensor * daily_price_returns))

                    # Store daily return
                    daily_returns[t] = np.clip(daily_return, -0.25, 0.25)  # Clip extreme values

                    # Update portfolio value
                    portfolio_values[t+1] = portfolio_values[t] * (1 + daily_return)

            except Exception as e:
                print(f"Error at period {period+1}: {e}")
                # Fill in weights and zeros for returns
                for t in range(period_start_idx, period_end_idx - 1):
                    if t < len(all_weights):
                        all_weights[t] = equal_weights
                        daily_returns[t] = 0.0

    # Store results in portfolio object
    portfolio.weights = all_weights
    portfolio.rets = daily_returns

    # Calculate portfolio statistics
    try:
        portfolio.stats()
        print(f"\nPerformance of Equal-Weighted Portfolio with {holding_period}-day rebalancing:")
        print(f"Mean Daily Return: {portfolio.mu:.4f}")
        print(f"Daily Volatility: {portfolio.vol:.4f}")
        print(f"Sharpe Ratio: {portfolio.sharpe:.4f}")

        # Calculate annualized metrics
        annual_factor = 252  # Trading days in a year
        annual_return = (1 + portfolio.mu)**annual_factor - 1
        annual_vol = portfolio.vol * np.sqrt(annual_factor)
        annual_sharpe = annual_return / annual_vol if annual_vol > 0 else 0

        print(f"\nAnnualized Metrics:")
        print(f"Annual Return: {annual_return:.4f}")
        print(f"Annual Volatility: {annual_vol:.4f}")
        print(f"Annual Sharpe Ratio: {annual_sharpe:.4f}")

    except Exception as e:
        print(f"Error in stats calculation: {e}")
        # Manual calculation
        returns = np.array(daily_returns)
        valid_returns = returns[returns != 0]  # Filter out zeros

        if len(valid_returns) > 0:
            mean_return = np.mean(valid_returns)
            std_return = np.std(valid_returns)

            print(f"Mean Daily Return: {mean_return:.4f}")
            print(f"Daily Volatility: {std_return:.4f}")

            if std_return > 0:
                print(f"Sharpe Ratio: {mean_return/std_return:.4f}")

                # Annualized metrics
                annual_factor = 252  # Trading days in a year
                annual_return = (1 + mean_return)**annual_factor - 1
                annual_vol = std_return * np.sqrt(annual_factor)
                annual_sharpe = annual_return / annual_vol if annual_vol > 0 else 0

                print(f"\nAnnualized Metrics:")
                print(f"Annual Return: {annual_return:.4f}")
                print(f"Annual Volatility: {annual_vol:.4f}")
                print(f"Annual Sharpe Ratio: {annual_sharpe:.4f}")
        else:
            print("No valid returns to calculate statistics")

    # Store portfolio values for further analysis
    portfolio.values = portfolio_values

    return portfolio

def compare_to_benchmark(portfolios, benchmark_portfolio, names, holding_period=14):
    """
    Compares model portfolios to an equal-weight benchmark

    Parameters:
    -----------
    portfolios : list
        List of portfolio objects from different models
    benchmark_portfolio : object
        The benchmark portfolio (equal weights)
    names : list
        Names of the model portfolios
    holding_period : int, optional
        Holding period in days

    Returns:
    --------
    dict
        Performance comparison between models and benchmark
    """
    # Get benchmark performance
    benchmark_return = benchmark_portfolio.mu
    benchmark_vol = benchmark_portfolio.vol
    benchmark_sharpe = benchmark_portfolio.sharpe

    # Prepare results
    results = {
        "Portfolio": ["Equal Weight Benchmark"] + names,
        "Mean Return": [benchmark_return],
        "Volatility": [benchmark_vol],
        "Sharpe Ratio": [benchmark_sharpe],
        "Excess Return vs Benchmark": [0.0],  # Benchmark compared to itself is 0
        "Information Ratio": [0.0]  # Benchmark compared to itself is 0
    }

    # Calculate metrics for each portfolio
    for i, portfolio in enumerate(portfolios):
        # Get portfolio performance
        model_return = portfolio.mu
        model_vol = portfolio.vol
        model_sharpe = portfolio.sharpe

        # Calculate excess return vs benchmark
        excess_return = model_return - benchmark_return

        # Calculate tracking error (standard deviation of excess returns)
        # First, we need to calculate excess returns for each period
        excess_rets = np.array(portfolio.rets) - np.array(benchmark_portfolio.rets)
        tracking_error = np.std(excess_rets)

        # Calculate information ratio
        information_ratio = excess_return / tracking_error if tracking_error > 0 else 0

        # Add to results
        results["Mean Return"].append(model_return)
        results["Volatility"].append(model_vol)
        results["Sharpe Ratio"].append(model_sharpe)
        results["Excess Return vs Benchmark"].append(excess_return)
        results["Information Ratio"].append(information_ratio)

    # Print results
    print("\nPerformance Comparison:")
    print(f"{'Portfolio':<25} {'Mean Return':<12} {'Volatility':<12} {'Sharpe':<12} {'Excess Return':<15} {'Info Ratio':<12}")
    print("-" * 90)

    for i in range(len(results["Portfolio"])):
        print(f"{results['Portfolio'][i]:<25} {results['Mean Return'][i]:>8.4f}    {results['Volatility'][i]:>8.4f}    {results['Sharpe Ratio'][i]:>8.4f}    {results['Excess Return vs Benchmark'][i]:>10.4f}      {results['Information Ratio'][i]:>8.4f}")

    return results



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import matplotlib.gridspec as gridspec

class SimplePortfolioVisualizer:
    """
    A streamlined visualization tool for comparing multiple portfolio strategies.
    Designed for simplicity and easy-to-read comparisons.
    """

    def __init__(self, portfolios, names):
        """
        Initialize with portfolio objects.

        Parameters:
        -----------
        portfolios : list
            List of portfolio objects with weights, returns, and dates
        names : list
            List of names for each portfolio strategy
        """
        self.portfolios = portfolios
        self.names = names

        # Use distinct colors for better differentiation
        self.colors = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099']

        # Set up basic style
        plt.style.use('default')
        self.set_plot_style()

    def asset_price_movement(self, Y_test=None, normalize=True):
        """
        Plot the underlying assets' price movements during the test period.

        Parameters:
        -----------
        Y_test : pd.DataFrame or similar
            Asset price data with dates as index and assets as columns
        normalize : bool
            Whether to normalize prices to start from 1.0

        Returns:
        --------
        matplotlib.figure.Figure
        """
        if Y_test is None:
            raise ValueError("Y_test data must be provided to plot asset price movements")

        fig, ax = plt.subplots(figsize=(12, 6))

        # Get asset values
        prices = Y_test.values
        dates = Y_test.index

        # Get asset names if available, otherwise use indices
        if hasattr(Y_test, 'columns'):
            asset_names = Y_test.columns
        else:
            asset_names = [f"Asset {i+1}" for i in range(prices.shape[1])]

        # Normalize prices if requested
        if normalize:
            prices = prices / prices[0, :] if len(prices) > 0 else prices

        # Colormap for assets
        colors = plt.cm.tab20(np.linspace(0, 1, len(asset_names)))

        # Plot each asset
        for i in range(len(asset_names)):
            ax.plot(dates, prices[:, i], label=asset_names[i],
                   color=colors[i], linewidth=1.5, alpha=0.8)

        # Format plot
        title = 'Normalized Asset Price Movements' if normalize else 'Asset Price Movements'
        ax.set_title(title, fontweight='bold')
        ax.set_xlabel('Date')
        ax.set_ylabel('Price' if not normalize else 'Normalized Price')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        plt.xticks(rotation=45)

        # Add legend with smaller font size if many assets
        if len(asset_names) > 10:
            ax.legend(loc='upper left', fontsize=8, ncol=2)
        else:
            ax.legend(loc='best')

        plt.tight_layout()
        return fig

    def annual_sharpe_ratio(self, risk_free_rate=0.0):
        """
        Create bar chart of Sharpe ratios by year for each portfolio model.

        Parameters:
        -----------
        risk_free_rate : float, optional
            Risk-free rate (daily/period rate, not annual)

        Returns:
        --------
        matplotlib.figure.Figure
        """
        fig, ax = plt.subplots(figsize=(12, 6))

        # Group by year for each portfolio and calculate metrics
        yearly_sharpe = {}

        for i, portfolio in enumerate(self.portfolios):
            model_name = self.names[i]
            yearly_sharpe[model_name] = {}

            # Get returns and dates
            returns = np.array(portfolio.rets)
            dates = np.array(portfolio.dates)

            # Extract period returns if 2D array
            if len(returns.shape) > 1 and returns.shape[1] > 0:
                period_returns = returns[:, 0]
            else:
                period_returns = returns

            # Get years from dates (handle different date types)
            years = []
            for d in dates:
                if isinstance(d, np.datetime64):
                    # For numpy.datetime64, convert to string and extract year
                    year_str = str(d).split('-')[0]
                    years.append(int(year_str))
                elif hasattr(d, 'year'):
                    # For datetime.datetime or pandas Timestamp
                    years.append(d.year)
                else:
                    # For other formats, try converting to datetime first
                    try:
                        date_obj = pd.to_datetime(d)
                        years.append(date_obj.year)
                    except:
                        # If all else fails, just use a placeholder
                        years.append(0)

            years = np.array(years)
            unique_years = np.unique(years)

            # Calculate Sharpe ratio for each year
            for year in unique_years:
                if year == 0:  # Skip invalid years
                    continue

                year_mask = years == year
                year_returns = period_returns[year_mask]

                # Only calculate if we have enough data
                if len(year_returns) > 10:  # Arbitrary threshold
                    mean_return = np.mean(year_returns)
                    std_return = np.std(year_returns)

                    # Sharpe ratio (avoid division by zero)
                    sharpe = (mean_return - risk_free_rate) / std_return if std_return > 0 else 0

                    yearly_sharpe[model_name][year] = sharpe

        # Prepare data for plotting
        all_years = sorted(set().union(*[set(d.keys()) for d in yearly_sharpe.values()]))

        # Set up bar positions
        bar_width = 0.8 / len(self.portfolios)
        positions = np.arange(len(all_years))

        # Plot bars for each model
        for i, model_name in enumerate(self.names):
            model_sharpe = [yearly_sharpe[model_name].get(year, np.nan) for year in all_years]
            model_positions = positions + i * bar_width - 0.4 + bar_width / 2

            ax.bar(model_positions, model_sharpe, width=bar_width,
                  color=self.colors[i % len(self.colors)], alpha=0.7,
                  label=model_name)

        # Format plot
        ax.set_title('Annual Sharpe Ratios by Portfolio Model', fontweight='bold')
        ax.set_xlabel('Year')
        ax.set_ylabel('Sharpe Ratio')
        ax.set_xticks(positions)
        ax.set_xticklabels(all_years)

        # Add zero line for reference
        ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.7)

        # Add legend
        ax.legend(loc='best')

        plt.tight_layout()
        return fig


    def set_plot_style(self):
        """Set clean style for all plots"""
        plt.rcParams['figure.figsize'] = (10, 6)
        plt.rcParams['font.size'] = 12
        plt.rcParams['axes.labelsize'] = 14
        plt.rcParams['axes.titlesize'] = 16
        plt.rcParams['xtick.labelsize'] = 12
        plt.rcParams['ytick.labelsize'] = 12
        plt.rcParams['legend.fontsize'] = 12
        plt.rcParams['axes.spines.top'] = False
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.grid'] = True
        plt.rcParams['grid.alpha'] = 0.3

    
    def create_dashboard(self, output_file=None):
        """
        Create a simple, easy-to-read dashboard with the essential visualizations.

        Parameters:
        -----------
        output_file : str, optional
            File path to save the dashboard image

        Returns:
        --------
        matplotlib.figure.Figure
        """
        # Create figure with a 2x2 grid
        fig = plt.figure(figsize=(16, 14))
        gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1])

        # 1. Cumulative Returns Plot (top left)
        ax1 = fig.add_subplot(gs[0, 0])

        for i, portfolio in enumerate(self.portfolios):
            # Check the shape of returns to determine how to handle them
            returns = np.array(portfolio.rets)
            dates = np.array(portfolio.dates)

            # Check if returns is 2D array and handle appropriately
            if len(returns.shape) > 1 and returns.shape[1] > 1:
                # If returns has multiple columns, second column might be cumulative returns
                if returns.shape[1] == 2:
                    # Use the cumulative returns directly (second column)
                    cum_returns = returns[:, 1]
                else:
                    # Use the first column as period returns and calculate cumulative
                    period_returns = returns[:, 0]
                    cum_returns = np.cumprod(1 + period_returns)
            else:
                # Traditional 1D array of returns
                cum_returns = np.cumprod(1 + returns)

            ax1.plot(dates, cum_returns,
                    color=self.colors[i % len(self.colors)],
                    linewidth=2, label=self.names[i])

        ax1.set_title('Cumulative Returns (Growth of $1)', fontweight='bold')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Value')
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        ax1.legend(loc='best')

        # 2. Risk-Return Scatter (top right)
        ax2 = fig.add_subplot(gs[0, 1])

        returns_data = []
        volatility_data = []
        sharpe_ratios = []

        for i in range(len(self.portfolios)):
            returns = np.array(self.portfolios[i].rets)

            # Extract period returns if returns is a 2D array
            if len(returns.shape) > 1 and returns.shape[1] > 0:
                period_returns = returns[:, 0]  # First column
            else:
                period_returns = returns

            mean_return = np.mean(period_returns) * 100 *14 # return over 14 days
            volatility = np.std(period_returns) * 100
            sharpe = mean_return / volatility if volatility > 0 else 0

            returns_data.append(mean_return)
            volatility_data.append(volatility)
            sharpe_ratios.append(sharpe)

        # Plot each portfolio
        for i in range(len(self.portfolios)):
            ax2.scatter(volatility_data[i], returns_data[i],
                       color=self.colors[i % len(self.colors)],
                       s=120, alpha=0.7,
                       label=f"{self.names[i]} (SR={sharpe_ratios[i]:.2f})")

            ax2.annotate(self.names[i],
                        xy=(volatility_data[i], returns_data[i]),
                        xytext=(7, 0), textcoords="offset points",
                        fontsize=12)

        ax2.set_title('Risk-Return Analysis', fontweight='bold')
        ax2.set_xlabel('Volatility (%)')
        ax2.set_ylabel('Return (%)')
        #ax2.legend(loc='upper left')

        # 3. Drawdown Chart (bottom left)
        ax3 = fig.add_subplot(gs[1, 0])

        for i, portfolio in enumerate(self.portfolios):
            returns = np.array(portfolio.rets)
            dates = np.array(portfolio.dates)

            # Handle returns based on shape
            if len(returns.shape) > 1 and returns.shape[1] > 1:
                if returns.shape[1] == 2:
                    # If we have cumulative returns, use them directly
                    cum_returns = returns[:, 1]
                else:
                    # Otherwise calculate from period returns
                    period_returns = returns[:, 0]
                    cum_returns = np.cumprod(1 + period_returns)
            else:
                cum_returns = np.cumprod(1 + returns)

            # Calculate drawdowns
            running_max = np.maximum.accumulate(cum_returns)
            drawdowns = (cum_returns / running_max - 1) * 100

            ax3.plot(dates, drawdowns,
                    color=self.colors[i % len(self.colors)],
                    linewidth=2, label=self.names[i])

            # Add max drawdown point
            max_drawdown = np.min(drawdowns)
            max_dd_idx = np.argmin(drawdowns)

            if max_dd_idx < len(dates):
                ax3.scatter(dates[max_dd_idx], drawdowns[max_dd_idx],
                           color=self.colors[i % len(self.colors)], s=80, zorder=5)

                ax3.annotate(f"{max_drawdown:.1f}%",
                            xy=(dates[max_dd_idx], drawdowns[max_dd_idx]),
                            xytext=(7, -10), textcoords="offset points",
                            color=self.colors[i % len(self.colors)], fontsize=10)

        ax3.set_title('Portfolio Drawdowns', fontweight='bold')
        ax3.set_xlabel('Date')
        ax3.set_ylabel('Drawdown (%)')
        ax3.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        ax3.legend(loc='lower right')

        # 4. Performance Table (bottom right)
        ax4 = fig.add_subplot(gs[1, 1])
        ax4.axis('off')

        # Calculate performance metrics
        metrics = []
        metric_names = [
            'Mean Return (%)',
            'Volatility (%)',
            'Sharpe Ratio',
            'Max Drawdown (%)',
            'Win Rate (%)'
        ]

        for i in range(len(self.portfolios)):

            returns = np.array(self.portfolios[i].rets)
            dates = np.array(self.portfolios[i].dates)
            # Handle returns based on shape
            if len(returns.shape) > 1 and returns.shape[1] > 1:
                if returns.shape[1] == 2:
                    # If we have cumulative returns, use them directly
                    cum_returns = returns[:, 1]
                else:
                    # Otherwise calculate from period returns
                    period_returns = returns[:, 0]
                    cum_returns = np.cumprod(1 + period_returns)
            else:
                cum_returns = np.cumprod(1 + returns)

            # Example cum_returns (cumulative returns)
            cum_returns = np.array(cum_returns)


            # Periodic returns from cumulative returns
            returns = np.diff(cum_returns) / cum_returns[:-1]

            # Final return
            final_return = np.mean(returns)*100 * 14  # return over 14 days

            # Volatility (standard deviation of returns)
            volatility = np.std(returns, ddof=1)*100

            # Sharpe ratio (assuming risk-free rate is 0 for simplicity)
            sharpe = final_return / volatility if volatility != 0 else np.nan

            # Max drawdown
            running_max = np.maximum.accumulate(cum_returns)
            drawdowns = (cum_returns - running_max) / running_max
            max_drawdown = drawdowns.min()

            # Win rate (percentage of positive returns)
            win_rate = np.mean(returns > 0)
            metrics.append([
                final_return,  
                volatility,   
                sharpe,
                max_drawdown,
                win_rate
            ])

        # Create table data
        cell_text = []
        for i, metric_values in enumerate(metrics):
            row = [f"{value:.2f}" for value in metric_values]
            cell_text.append(row)

        cell_text = np.array(cell_text).T

        # Create table
        table = ax4.table(
            cellText=cell_text,
            rowLabels=metric_names,
            colLabels=self.names,
            loc='center',
            cellLoc='center'
        )

        # Style table
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1, 1.5)

        # Style headers
        for i, name in enumerate(self.names):
            cell = table[(0, i)]
            cell.set_text_props(weight='bold', color='white')
            cell.set_facecolor(self.colors[i % len(self.colors)])

        # Style row labels
        for i, name in enumerate(metric_names):
            cell = table[(i+1, -1)]
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#f0f0f0')

        ax4.set_title('Performance Summary', fontsize=16, fontweight='bold', pad=20)

        # Add main title
        plt.suptitle('Portfolio Performance Comparison', fontsize=20, fontweight='bold', y=0.98)

        plt.tight_layout(rect=[0, 0, 1, 0.96])

        if output_file:
            plt.savefig(output_file, dpi=300, bbox_inches='tight')

        return fig


'\n# Create visualizer with all 4 portfolios\nvisualizer = SimplePortfolioVisualizer(\n    [equal_portfolio, standard_markowitz_portfolio, max_sharpe_portfolio, risk_budget_portfolio],\n    ["Equal Weight", "Standard Markowitz", "Max Sharpe", "Risk Budget"]\n)\n\n# Create the dashboard\ndashboard = visualizer.create_dashboard(output_file="portfolio_comparison.png")\n'

In [ ]:
%cd /content/drive/MyDrive/5370_Project_E2E/E2E-DRO/

# Distributionally Robust End-to-End Portfolio Construction
# Experiment 1 - General
####################################################################################################
# Import libraries
####################################################################################################
import torch
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
plt.close("all")
plt.rcParams['text.usetex'] = False


# Make the code device-agnostic
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Import E2E_DRO functions
from e2edro import e2edro_Markov as e2e
from e2edro import DataLoad as dl
from e2edro import BaseModels as bm
from e2edro import PlotFunctions as pf

# Path to cache the data, models and results
cache_path = "./cache/exp_final/"


#---------------------------------------------------------------------------------------------------
# Initialize parameters
#---------------------------------------------------------------------------------------------------
perf_loss='sharpe_loss'
perf_period = 13
pred_loss_factor = 0.5
prisk = 'p_var'
dr_layer = 'hellinger'
lr_list = [0.0125]# [0.005, 0.0125, 0.02]
epoch_list = [40]#[30, 40, 50, 60, 80, 100]
set_seed = 1000
use_cache = False

# ---------------  DATA Collection For Price Predict -----------------
import pandas as pd
import numpy as np
import yfinance as yf
import time

def fetch_data(tickers, start, end, freq='1d', use_cache=False, save_results=False, cache_path='./cache/original_close.pkl'):
    if use_cache:
        data = pd.read_pickle(cache_path)
    else:
        data = pd.DataFrame()
        batch_size = 5

        for i in range(0, len(tickers), batch_size):
            batch = tickers[i:i+batch_size]
            try:
                print(f"Downloading batch: {batch}")
                batch_data = yf.download(batch, start=start, end=end, interval=freq, progress=False, group_by='ticker')

                # Handle different format if downloading multiple or single tickers
                if len(batch) == 1:
                    ticker = batch[0]
                    batch_data = batch_data['Close'].to_frame(name=ticker)
                else:
                    batch_data = {ticker: batch_data[ticker]['Close'] for ticker in batch if ticker in batch_data.columns.levels[0]}
                    batch_data = pd.DataFrame(batch_data)

                data = pd.concat([data, batch_data], axis=1)
            except Exception as e:
                print(f"Failed to download {batch}: {e}")
            time.sleep(5)  # pause to avoid rate limiting

        if save_results:
            data.to_pickle(cache_path)

    return data

def create_features(data,tickers,lag=5,windows=[10,20,30]):
    ret = data.pct_change()
    ticker2feature = {}
    for ticker in tickers:
        df = pd.DataFrame(index=ret.index)
        df['close'] = data[ticker]
        df['ret'] = ret[ticker]  # actual ret

        for l in range(1,lag+1):
            df[f'ret_lag_{l}'] = ret[ticker].shift(l)
        for w in windows:
            df[f'ret_mean_{w}d'] = ret[ticker].rolling(w).mean()
        for w in windows:
            df[f'ret_vol_{w}d'] = ret[ticker].rolling(w).std()

        ticker2feature[ticker] = df
    features = pd.concat(ticker2feature.values(), axis=1, keys=ticker2feature.keys())
    return features.dropna()
tickers = ["VTI","IWM","AGG","LQD","MUB","DBC","GLD"]
# tickers = ['AAPL', 'MSFT', 'AMZN', 'C', 'JPM', 'BAC', 'XOM', 'HAL', 'MCD', 'WMT',
#         'COST', 'CAT', 'LMT', 'JNJ', 'PFE', 'DIS', 'VZ', 'T', 'ED', 'NEM']
freq = '1d'
start = '2010-01-01'
end = '2021-09-30'
# Train, validation and test split percentage
split = [0.6, 0.4]

# Number of observations per window
n_obs = 30

# Number of features and assets
Y_data = fetch_data(tickers, start, end, freq='1d',use_cache=True, save_results=False)
X_data = create_features(Y_data, tickers)
Y_data = Y_data.iloc[30:]



class TimeSeriesDataFrame:
    def __init__(self, data, n_obs, split=None):
        self.data = data  # Store data as a pandas DataFrame
        self.n_obs = n_obs  # Store number of observations for windowing
        self.split = split  # Store the train/test split (if provided)

        if self.split:
            self.train_data, self.test_data = self.train_test_split()

        # Set attributes for train and test data directly
        self.train = self.train_data
        self.test = self.test_data
    def train_test_split(self):
        split_idx = int(len(self.data) * self.split[0])  # Calculate the index for the split
        train_data = self.data.iloc[:split_idx]  # Train data is the first portion
        test_data = self.data.iloc[split_idx:]   # Test data is the remaining portion
        return train_data, test_data

    def split_update(self, split):
        self.split = split
        self.train_data, self.test_data = self.train_test_split()
        self.train = self.train_data
        self.test = self.test_data

split = [0.6, 0.4]

X = TimeSeriesDataFrame(X_data, n_obs=n_obs, split=split)
Y = TimeSeriesDataFrame(Y_data, n_obs=n_obs, split=split)
n_x, n_y = X.data.shape[1], Y.data.shape[1]

/content/drive/MyDrive/5370_Project_E2E/E2E-DRO


In [ ]:
# load all the models

from e2edro import e2edro_Risk as e2e

with open(cache_path+'risk_budget.pkl', 'rb') as inp:
        risk_budget = pickle.load(inp)
risk_budget_port = risk_budget.non_overlapping_risk_test(X, Y, 14)

with open(cache_path+'min_variance.pkl', 'rb') as inp:
        min_variance = pickle.load(inp)
min_variance_e2e_port = min_variance.non_overlapping_risk_test(X, Y, 14)

from e2edro import e2edro_Markov as e2e
with open(cache_path+'min_var.pkl', 'rb') as inp:
        min_var = pickle.load(inp)
min_variance_port = min_var.non_overlapping_test(X, Y, 14)

# Create a new model with the same architecture
maxSharp_markov_new = e2e.e2e_net(n_x, n_y, n_obs, prisk=prisk,
                    train_pred=True, train_gamma=False, train_delta=False,
                    set_seed=set_seed, opt_layer='max_sharpe', perf_loss=perf_loss,
                    perf_period=perf_period, pred_loss_factor=pred_loss_factor, variant = "max_sharpe").double()

# Load the saved state - with weights_only=False
maxSharp_markov_new.load_state_dict(torch.load(cache_path+'maxSharp_markov_state.pt', weights_only=False))

max_sharpe_port = maxSharp_markov_new.non_overlapping_test(X, Y, 14)

with open(cache_path+'standard_markov.pkl', 'rb') as inp:
        standard_markov = pickle.load(inp)
standard_mark_port = standard_markov.non_overlapping_test(X, Y, 14)


# Run equal weight test (no model needed)
equal_portfolio = non_overlapping_equal_weight_test(
    X=X,
    Y=Y,
    n_assets=7,  # Specify if known, otherwise it will be inferred from Y
    holding_period=14
)

In [ ]:
# Create visualizer with Markovwitze Portfolios
visualizer = SimplePortfolioVisualizer(
    [equal_portfolio, standard_mark_port, max_sharpe_port],
    ["Equal Weight", "Mean Var", "Max Shar"]
)

# Create the dashboard
dashboard = visualizer.create_dashboard(output_file="portfolio_comparison.png")
plt.show()

In [ ]:
#Plot the asset price movements
asset_price_plot = visualizer.asset_price_movement(Y_test=Y.test, normalize=True)

In [ ]:
# Create visualizer with Risk portfolios
visualizer_risk = SimplePortfolioVisualizer(
    [equal_portfolio, min_variance_e2e_port, min_variance_port, risk_budget_port],
    ["Equal Weights","E2E Min Var", "Min Var", "Risk Budget"]
)

# Create the dashboard
dashboard_risk = visualizer_risk.create_dashboard(output_file="portfolio_comparison.png")

In [ ]:
# Create visualizer with all portfolios
visualizer_risk = SimplePortfolioVisualizer(
    [equal_portfolio, min_variance_e2e_port, min_variance_port, risk_budget_port,standard_mark_port, max_sharpe_port],
    ["EW","e2e Min V", "Min V", "RB", "Mean V", "Max Shar"]
)

# Create the dashboard
dashboard_risk = visualizer_risk.create_dashboard(output_file="portfolio_comparison.png")